In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.wkt import loads

# 신천 교량별 15분 영향권 폴리곤 데이터

In [3]:
#데이터 읽어오기
df = pd.read_csv('D:/jupiter_workspace/intern/dip_intern/sincheon/02population/bridge_polygon.csv')
df = df[['name', 'geometry']]
df.columns = ['bridge', 'geometry']

#geometry 열 polygon type으로 생성
def load_valid(geo):
    try:
        return loads(geo)
    except:
        return np.nan
df.geometry = df.geometry.apply(load_valid)

gdf_sc = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:4326')
gdf_sc.head()

,bridge,geometry
0,가창,"POLYGON ((128.58400 35.89977, 128.58291 35.900..."
1,경대칠성,"POLYGON ((128.60581 35.86899, 128.60210 35.869..."
2,대봉희망,"POLYGON ((128.61472 35.84350, 128.61354 35.843..."
3,도청경대,"POLYGON ((128.59590 35.88019, 128.59317 35.883..."
4,동신수성,"POLYGON ((128.61729 35.85601, 128.60014 35.859..."


# 네이버, 구글, 트립어드바이저 명소 리뷰 데이터

In [21]:
def getScSight(site, lon, lat, name) :
    """
    해당 site 리뷰데이터 csv를 읽어와서 신천 영향권 안에 있는 곳에 대한 리뷰만 추출
    
    params
    ------
    site : 구글, 네이버, 트립어드바이저 (str)
    lon : df 내에서 경도(128.) 열의 이름 (str)
    lat : df 내에서 위도(38.) 열의 이름 (str)
    name : df 내에서 관광지명 열의 이름 (str)
    
    retruns
    ------
    df : 신천 영향권 내의 리뷰만 담은 데이터프레임 (columns: time, names, date, reviews, stars)
    """

    # 데이터 불러오기
    url = 'D:/jupiter_workspace/intern/dip_intern/sincheon/rawdata/20. 맛집, 호텔, 명소 리뷰데이터/' + site + '/' + site + '리뷰_대구 명소_외국인구분.csv'
    df = pd.read_csv(url, encoding='cp949')

    #관광지 데이터 gdf로 만들기
    df['geometry'] = df.apply(lambda row : Point([row[lon], row[lat]]), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='epsg:4326')

    #폴리곤 안에 있는 관광지만 추출
    scSight = gpd.sjoin(gdf_sc, gdf)
    scSight = scSight[['bridge', name, 'date', 'reviews', 'stars', lon, lat]]
    scSight.drop_duplicates([name, 'date', 'reviews', 'stars'], inplace=True)
    scSight = scSight.reset_index(drop=True)
    
    return scSight

In [22]:
scNaver = getScSight('네이버', 'lat', 'lng', 'names')
scNaver.head()

c:\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,bridge,names,date,reviews,stars,lat,lng
0,상동두산,공룡공원,2020.08.07,굿,3.5,128.603395,35.829925
1,상동두산,공룡공원,2020.08.07,굿,3.0,128.603395,35.829925
2,칠성신천,국채보상운동기념공원,2020.08.31,좋아요,5.0,128.602750,35.869007
3,칠성신천,국채보상운동기념공원,2020.06.07,주차장 이용했습니다아,5.0,128.602750,35.869007
4,칠성신천,국채보상운동기념공원,2020.02.01,대구를 상징하는 좋아요,3.5,128.602750,35.869007


In [23]:
scGoogle = getScSight('구글', 'lng', 'lat', 'name')
scGoogle.head()

c:\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,bridge,name,date,reviews,stars,lng,lat
0,경대칠성,꽃보라동산,2019/12/29,벗꽃이 이뻐용^^,5,128.599724,35.890023
1,경대칠성,꽃보라동산,2019/12/29,봄철 벚꽃이 만개 했을때 구경하기 딱 좋다. 그 외에는 별로다.,3,128.599724,35.890023
2,경대칠성,꽃보라동산,2019/12/29,다른 공원에 비해 식재한 것이 자연스럽다 보니 공감이 많이 갔다,5,128.599724,35.890023
3,경대칠성,꽃보라동산,2019/12/29,벚꽃 피고 1주일이면 꽃이 다 떨어져서 별다른 재미가 없다,3,128.599724,35.890023
4,경대칠성,꽃보라동산,2019/12/29,저녁 노을 질때 구름다리에서 신천바라보는 풍경이 좋다!,5,128.599724,35.890023


In [24]:
scTrip = getScSight('트립어드바이저', 'lat', 'lng', 'name')
scTrip.head()

c:\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,bridge,name,date,reviews,stars,lat,lng
0,경대칠성,대구도시철도,2019년 10월,"빠르고 효율적인탐색하기가 쉽고, 사방에 표지판과지도가 있습니다 (한국어를 읽을 수 ...",5.0,128.601445,35.871435
1,경대칠성,대구도시철도,2019년 2월,"깨끗하고 안전해요거의 매일 이용하는데, 정확하고 좋아요. 청결하고, 환승 시간도 정...",5.0,128.601445,35.871435
2,경대칠성,대구도시철도,2016년 2월,대구의 중심 중앙로역!대구 지하철 1호선 중앙로역입니다~ ㅎㅎㅎ 나가는 출구가 많...,4.0,128.601445,35.871435
3,경대칠성,대구도시철도,2016년 4월,대구 중심가역반월당역과 함께 사람들이 많이 내리는 대구의 중심역이며 1호선입니다. ...,3.0,128.601445,35.871435
4,경대칠성,대구도시철도,2019년 10월,혼잡하지만 사용하기 쉬운아시아와 한국의 대부분의 경우와 마찬가지로 특히 지하철이 혼...,4.0,128.601445,35.871435


In [18]:
#네이버 df 양식 맞춰주기
scNaver['date'] = pd.to_datetime(scNaver['date'], format="%Y.%m.%d").dt.strftime('%Y-%m')
scNaver = scNaver.rename(columns={'names' : 'name', 'lng' : 'lat', 'lat' : 'lng'})
scNaver['stars'] = scNaver['stars'].astype(int)
scNaver = scNaver.dropna()

#구글 df 양식 맞춰주기
scGoogle['date'] = pd.to_datetime(scGoogle['date'], format="%Y/%m/%d").dt.strftime('%Y-%m')
scGoogle = scGoogle.dropna()

#트립어드바이저 df 양식 맞춰주기
scTrip['date'] = pd.to_datetime(scTrip['date'], format=" %Y년 %m월").dt.strftime('%Y-%m')
scTrip = scTrip.rename(columns={'lng' : 'lat', 'lat' : 'lng'})
scTrip['stars'] = scTrip['stars'].astype(int)
scTrip = scTrip.dropna()

#concat
scSight = pd.concat([scNaver, scGoogle, scTrip]).reset_index(drop=True)
scSight

,bridge,name,date,reviews,stars,lng,lat
0,상동두산,공룡공원,2020-08,굿,3,128.603395,35.829925
1,상동두산,공룡공원,2020-08,굿,3,128.603395,35.829925
2,칠성신천,국채보상운동기념공원,2020-08,좋아요,5,128.602750,35.869007
3,칠성신천,국채보상운동기념공원,2020-06,주차장 이용했습니다아,5,128.602750,35.869007
4,칠성신천,국채보상운동기념공원,2020-02,대구를 상징하는 좋아요,3,128.602750,35.869007
...,...,...,...,...,...,...,...
3225,동신수성,김광석 길,2017-09,김광석대구 1일 도시투어하면서 가본장소. 시내에 위치하고있어서 가기편했습니다. 학생...,3,128.607223,35.861241
3226,동신수성,김광석 길,2017-05,김광석거리시장옆에 위치해 있고 거리가 길지는 않으나 김광석 노래와 모습으로 벽화가 ...,4,128.607223,35.861241
3227,상동두산,수성아쿠아수영장,2016-10,"초등학교 내 수영장입니다.수성못에 대해서 리뷰 쓰신 분이 많은데, 수성아쿠아수영장은...",5,128.610227,35.836837
3228,상동두산,수성아쿠아수영장,2015-10,대구에서 가볼만한 곳아빠 어디가에서 정웅인네가 여기서 놀아서 조금 유명해졌고요. 오...,4,128.610227,35.836837


In [19]:
scSight.to_csv('20sight_new.csv')

In [25]:
scSight.name.unique()

array(['공룡공원', '국채보상운동기념공원', '꽃보라동산', '대현어린이공원', '대구 삼성 창조 캠퍼스', '칠성시장',
       '대구제일교회', '건들바위', '방천시장', '달구벌대종', '김광석 다시 그리기 길', '용두산', '들안길공원',
       '건들바위(대구입암)', '아르떼 수성랜드', '수성못', '구암서원 (영남선비문화수련원) 숭현사', '연암공원',
       '침산공원', '대구도시철도', '김광석 길', '수성아쿠아수영장'], dtype=object)